## Particle swarm optimization (PSO)

### Setup

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from paper import PSO, Mode

plt.rcParams["figure.dpi"] = 100

### Intro

#### Papers

[Particle Swarm Optimization](https://www.cs.tufts.edu/comp/150GA/homeworks/hw3/_reading6%201995%20particle%20swarming.pdf) - Kennedy and Eberhart 1995

[A Modified Particle Swarm Optimizer](https://www.researchgate.net/profile/Yuhui-Shi/publication/3755900_A_Modified_Particle_Swarm_Optimizer/links/54d9a9700cf24647581f009e/A-Modified-Particle-Swarm-Optimizer.pdf?origin=publication_detail) - Shi and Eberhart 1998

#### Motivation
* Method was discovered through the **simulation of a social behavior model**
    
    > <br /> “In theory at least, individual members of the school can **profit** from the discoveries and previous experience of **all other members** of the school during the search for food. This advantage can become decisive, **outweighing the disadvantages** of competition for food items, whenever the resource is unpedictably distributed in patches"
    >
    > \- sociobiologist E. O. Wilson in reference to fish schooling<br />&nbsp;
        
* Allows optimization of **continuous nonlinear functions** without the need for them to be **convex or differentiable**

* Proved to be effective **training neural network weights** as well as optimizing **genetic algorithm test functions** such as Schaffer's f6 

##### Basic idea

> <br /> The algorithm works by having a **population** (aka swarm) of **candidate solutions** (aka particles). 
>
> These particles are moved around in the search-space based on their **own best-known position** as well as the **entire swarm's best known position**. 
> 
> By repeating these movements it is hoped, but **not guaranteed**, that a satisfactory solution will eventually be discovered. [ref](https://en.wikipedia.org/wiki/Particle_swarm_optimization#Algorithm)<br />&nbsp;

### Development of algorithm


#### Nearest Neighbor Velocity Matching and Craziness

Population randomly initialized on a torus pixel grid with both x and y velocities



##### Nearest Neighbor Velocity Matching

* at each iteration → each agent is assigned the velocity of its nearest neighbor
        
* cons → quickly settled on a unanimous, unchanging direction

In [ ]:
pso = PSO(Mode.NNVM, particles=2)
pso.run()

##### Craziness

* at each iteration → some change was added to random chosen X and Y velocities
<br/><br/>&nbsp;&nbsp;&nbsp;&nbsp;- doubt → `fixed` change to `some` velocities or `random` change to `all` velocities?

* gave the simulation a "lifelike" appearance though the variation was wholly artificial

In [ ]:
pso = PSO(Mode.NNVM_CRAZINESS, particles=16, duration=1000)
pso.run()

#### The cornfield vector
    
* Heppner's bird simulations had a dynamic force
    
    * birds flocked around a "roost" → position on the pixel screen that attracted them until they landed there
    
    * removes the need for `craziness`
* A two-dimensional vector of XY coordinates on the pixel plane was introduced → i.e function to be optimized

    $$Eval = \sqrt{(presentx-100)^2}+\sqrt{(presenty-100)^2}$$

    * Rule to change velocity
        
        * Each agent "remembers" best value and the (x,y) that produces it
        
            $$v_x[\space] = \begin{cases}
                v_x[\space] - rand()*p_{inc} &\text{if } current_x[\space] > pbestx[\space]  \\
                v_x[\space] + rand()*p_{inc} &\text{if } current_x[\space] < pbestx[\space]
            \end{cases} $$

        * Each agent “knows” the global best position and its value
        
            $$v_x[\space] = \begin{cases}
                v_x[\space] - rand()*g_{inc} &\text{if } current_x[\space] > pbestx[gbest]  \\
                v_x[\space] + rand()*g_{inc} &\text{if } current_x[\space] < pbestx[gbest]
            \end{cases}$$

In [ ]:
pso = PSO(Mode.CORNFIELD_VECTOR, particles=16, p_inc=1, g_inc=1, duration=1500)
pso.run()

#### Eliminating ancillary variables

* `craziness` was removed → algorithm looks just as realistic without it

* `nnvm` was removed → optimization occured slightly faster

* variables $pbest$ and $gbest$ and their increments are both necessary

* equal values of $p_{inc}$ and $g_{inc}$ seem to result in most effective search

In [ ]:
pso = PSO(Mode.ONLY_CORNFIELD, particles=16, p_inc=0.1, g_inc=0.1)
pso.run()

#### Multidimensional search

* changed from one dimensional arrays to $DxN$ matrices

##### Experiments

* `3-layer NN` (2-3-1) solving the exclusive-or (XOR) problem
    
    * particles flying 13-dimensional space until an **average sum-squared error per PE** criterion was met
        
        \- doubt → calculated sum-squared error for each node and then the average had to meet certain criterion? why not just the last node?             

    * trained to an $e$ < 0.05 criterion, in an average of **30.7 iterations** with **20 agents**

* neural network to `classify` the Fisher Iris dataset

    * over 10 training sessions → average of 284 epochs was required
<br/><br/>

* weights found by particle swarms sometimes `generalize better` than solutions found by gradient descent

    * example dataset of electroencephalograms → 89% correct GD vs 92% correct PS
    
        \- doubt → correct based on what metric?

* benchmark with genetic algorithms found in Davis → the extremely nonlinear Schaffer f6 function
    
    * couldn't find the Davis and Schaffer f6 reference but the function somehow appeared in some websites

#### Acceleration by distance

- velocity adjustments were based on a crude inequality test
- velocities are not adjusted according to their difference per dimension from best locations
    
    $$
    v_x[\space][\space] = v_x[\space][\space] + rand()*p_{inc}*(pbest_x[\space][\space]- current_x[\space][\space])
    $$

#### Current simplified version
- no good way to guess if $p_{inc}$ or $g_{inc}$ should be larger → removed from algorithm

- stochastic factor multiplied by 2 to give a mean of 1

$$
v_x[\space][\space] = v_x[\space][\space] + 2 * rand() * (pbest_x[\space][\space] - current_x[\space][\space]) + 2 * rand() * (pbest_x[\space][gbest] - current_x[\space][\space])
$$


In [ ]:
pso = PSO(Mode.CURRENT, particles=16)
pso.run()

### Some alternatives

#### ❌ reduce $pbest$ and $gbest$ terms into one
- agent is propelled toward a weighted average of the two “best” points

- converges to that point whether optimum or not

- two stochastic “kicks” are needed

#### ❌ two types of agents → explorers and settlers
- explorers used the inequality test while settlers used the difference term

- hypothesis was that explorers would extrapolate outside the “known” region and settlers would micro-explore regions that had found to be good

#### ❌ remove momentum of $v_x[\space][\space]$
- ineffective at finding global optima
#### ✅ inertia weight $w$ is added to balance global and local search (from extra paper)
$$
v_x[\space][\space] = w * v_x[\space][\space] + 2 * rand() * (pbest_x[\space][\space] - current_x[\space][\space]) + 2 * rand() * (pbest_x[\space][gbest] - current_x[\space][\space])
$$
* small $w$ → PSO behaves more like a **local** search algorithm
* big $w$ → PSO behaves more like a **global** search algorithm

* motivation
    * PSO without $v_x[\space][\space]$ → search space shrinks through generations → resembles local search
    * By adding $v_x[\space][\space]$ → particles tend to expand the search space → more like global search
    * as usual there is a tradeoff between global and local search




In [ ]:
pso = PSO(Mode.EXTRA, particles=16, p_inc=0.1, g_inc=0.1, w=0.9)
pso.run()

### Caveat

The algorithm as is, is `limited` to problems without restrictions.

A simple `workaround` could be to add the restrictions through the use of `Lagrange multipliers`, i.e multiplied by a constant in the objective.

However, since these papers were written there's probably lots of literature on how to tackle the problem effectively.

### Swarm theory

Five basic principles of swarm intelligence
|  | Name | Description |
| --- | --- | --- |
| 1. | Proximity principle | the population should be able to carry out **simple space and time computations** |
| 2. | Quality principle | the population should be able to **respond to quality factors** in the environment |
| 3. | Principle of diverse response | the population should not commit its activities along excessively narrow channels |
| 4. | Principle of stability | the population **should not change** its mode of behavior **every time** the environment changes |
| 5. | Principle of adaptability | the population **must be able to change** behavior mode when it's **worth the computational price** |

How particle swarm applies to them

|  | Name | Description |
| --- | --- | --- |
| 1. | Proximity principle | n-dimensional space calculations carried out over a series of time steps |
| 2. | Quality principle | the population is responding to quality factors $pbest$ and $gbest$ |
| 3. | Principle of diverse response | the allocation of responses between $pbest$ and $gbest$ ensures a diversity of response. |
| 4. | Principle of stability | the population changes its state **only** when $gbest$ changes |
| 5. | Principle of adaptability | the population **does** change when $gbest$ changes |

### Ending

> <br/>The term particle was selected as a **compromise**. While it could be argued that the population members are **mass-less** and **volume-less**, and thus could be called “points”, it is felt that velocities and accelerations are more appropriately applied to particles, even if each is defined to have arbitrarily **small mass and volume**.<br/>&nbsp;

> <br />The authors of this paper are a **social psychologist** and an **electrical engineer**. The particle swarm optimizer serves both of these fields equally well.
>
> Why is **social behavior** so ubiquitous in the animal kingdom? Because it **optimizes**. 
>
> What is a good way to solve engineering **optimization** problems? Modeling **social behavior**.<br/>&nbsp;

### Test functions

In [ ]:
from functions import ackley

pso = PSO(
    Mode.EXTRA,
    particles=16,
    grid_length=10,
    p_inc=0.1,
    g_inc=0.1,
    w=0.9,
    fitness_func=ackley,
    grid_center=np.array([0, 0]),
)
pso.run()

In [ ]:
from functions import schaffer_f6

pso = PSO(
    Mode.EXTRA,
    particles=16,
    grid_length=200,
    p_inc=1,
    g_inc=1,
    w=0.9,
    fitness_func=schaffer_f6,
    grid_center=np.array([0, 0]),
)
pso.run()

In [ ]:
from functions import himmelblau

pso = PSO(
    Mode.EXTRA,
    particles=16,
    grid_length=10,
    p_inc=1,
    g_inc=1,
    w=0.9,
    fitness_func=himmelblau,
    grid_center=np.array([0, 0]),
)
pso.run()